In [1]:
import pandas as pd
import numpy as np
import ast
import json
import requests
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns

from config import api_key

In [2]:
url = 'https://api.themoviedb.org/3/trending/movie/day?api_key='+api_key

In [3]:
r = requests.get(url)
r

<Response [200]>

In [4]:
json = r.json()
json

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/qK7Ssnrfvrt65F66A1thvehfQg2.jpg',
   'genre_ids': [16, 10751, 35, 12, 14, 9648],
   'id': 420821,
   'original_language': 'en',
   'original_title': "Chip 'n Dale: Rescue Rangers",
   'overview': 'Decades since their successful television series was canceled, Chip has succumbed to a life of suburban domesticity as an insurance salesman. Dale, meanwhile, has had CGI surgery and works the nostalgia convention circuit, desperate to relive his glory days. When a former cast mate mysteriously disappears, Chip and Dale must repair their broken friendship and take on their Rescue Rangers detective personas once again to save their friend’s life.',
   'poster_path': '/7UGmn8TyWPPzkjhLUW58cOUHjPS.jpg',
   'release_date': '2022-05-20',
   'title': "Chip 'n Dale: Rescue Rangers",
   'video': False,
   'vote_average': 7.3,
   'vote_count': 36,
   'popularity': 87.881,
   'media_type': 'movie'},
  {'adult': False,
   'backdrop_path': '

In [5]:
json.keys()

dict_keys(['page', 'results', 'total_pages', 'total_results'])

In [6]:
json['results']

[{'adult': False,
  'backdrop_path': '/qK7Ssnrfvrt65F66A1thvehfQg2.jpg',
  'genre_ids': [16, 10751, 35, 12, 14, 9648],
  'id': 420821,
  'original_language': 'en',
  'original_title': "Chip 'n Dale: Rescue Rangers",
  'overview': 'Decades since their successful television series was canceled, Chip has succumbed to a life of suburban domesticity as an insurance salesman. Dale, meanwhile, has had CGI surgery and works the nostalgia convention circuit, desperate to relive his glory days. When a former cast mate mysteriously disappears, Chip and Dale must repair their broken friendship and take on their Rescue Rangers detective personas once again to save their friend’s life.',
  'poster_path': '/7UGmn8TyWPPzkjhLUW58cOUHjPS.jpg',
  'release_date': '2022-05-20',
  'title': "Chip 'n Dale: Rescue Rangers",
  'video': False,
  'vote_average': 7.3,
  'vote_count': 36,
  'popularity': 87.881,
  'media_type': 'movie'},
 {'adult': False,
  'backdrop_path': '/rl49Qr4KbfTSYjXYAN8fMDIt2xS.jpg',
  'ge

In [7]:
type(json['results'])

list

In [8]:
type(json['results'][0])

dict

In [9]:
df = pd.DataFrame(json['results'])
df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,poster_path,release_date,title,video,vote_average,vote_count,popularity,media_type
0,False,/qK7Ssnrfvrt65F66A1thvehfQg2.jpg,"[16, 10751, 35, 12, 14, 9648]",420821,en,Chip 'n Dale: Rescue Rangers,Decades since their successful television seri...,/7UGmn8TyWPPzkjhLUW58cOUHjPS.jpg,2022-05-20,Chip 'n Dale: Rescue Rangers,False,7.3,36,87.881,movie
1,False,/rl49Qr4KbfTSYjXYAN8fMDIt2xS.jpg,"[28, 878, 14]",526896,en,Morbius,"Dangerously ill with a rare blood disorder, an...",/6JjfSchsU6daXk2AKX8EEBjO3Fm.jpg,2022-03-30,Morbius,False,6.3,938,6323.965,movie
2,False,/kiH3KPWi7BaRMvdAigcwrUFViHl.jpg,"[9648, 53, 28, 80]",818397,en,Memory,"Alex, an assassin-for-hire, finds that he's be...",/tXpLmtnArbGcE75iPJVYibW7BBf.jpg,2022-04-28,Memory,False,7.4,31,118.002,movie
3,False,/kB1Qrsg189eDz4Ej3ABXtLudClq.jpg,[35],810171,en,The Valet,World famous movie star Olivia faces a PR disa...,/q7FmdJHKMLIC4XgWfcFRIu2iVdL.jpg,2022-05-11,The Valet,False,6.3,12,68.677,movie
4,False,/wp3vpSWAIjKSEeYb8F5NSZfONqw.jpg,"[28, 12, 878, 35]",545611,en,Everything Everywhere All at Once,An aging Chinese immigrant is swept up in an i...,/w3LxiVYdWWRvEVdn5RYq6jIqkb1.jpg,2022-03-24,Everything Everywhere All at Once,False,8.7,254,325.871,movie
5,False,/egoyMDLqCxzjnSrWOz50uLlJWmD.jpg,"[28, 878, 35, 10751, 12]",675353,en,Sonic the Hedgehog 2,"After settling in Green Hills, Sonic is eager ...",/6DrHO1jr3qVrViUO6s6kFiAGM7.jpg,2022-03-30,Sonic the Hedgehog 2,False,7.8,1487,6201.709,movie
6,False,/xHrp2pq73oi9D64xigPjWW1wcz1.jpg,"[80, 9648, 53]",414906,en,The Batman,"In his second year of fighting crime, Batman u...",/74xTEgt7R36Fpooo50r9T25onhq.jpg,2022-03-01,The Batman,False,7.8,4672,2950.956,movie
7,False,/BlC5QrSHnEHFAjAYrLdngUIA8r.jpg,"[35, 99, 28]",828853,en,Jackass 4.5,"Through outrageous, never-before-seen footage,...",/kewHZSdRyOWaLwVwhmoMhjp40Lz.jpg,2022-05-20,Jackass 4.5,False,5.6,9,106.149,movie
8,False,/6fRVz7ZIZIGFEOq4THY2pdBBhfS.jpg,"[35, 10749]",934761,en,A Perfect Pairing,"To land a major client, an LA wine exec travel...",/h65lLhYEfRGOWVZzwX9n7vYOixf.jpg,2022-05-19,A Perfect Pairing,False,6.7,25,75.390,movie
9,False,/cqnVuxXe6vA7wfNWubak3x36DKJ.jpg,"[28, 12, 14, 53]",639933,en,The Northman,Prince Amleth is on the verge of becoming a ma...,/zhLKlUaF1SEpO58ppHIAyENkwgw.jpg,2022-04-07,The Northman,False,7.5,974,5772.331,movie


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              20 non-null     bool   
 1   backdrop_path      20 non-null     object 
 2   genre_ids          20 non-null     object 
 3   id                 20 non-null     int64  
 4   original_language  20 non-null     object 
 5   original_title     20 non-null     object 
 6   overview           20 non-null     object 
 7   poster_path        20 non-null     object 
 8   release_date       20 non-null     object 
 9   title              20 non-null     object 
 10  video              20 non-null     bool   
 11  vote_average       20 non-null     float64
 12  vote_count         20 non-null     int64  
 13  popularity         20 non-null     float64
 14  media_type         20 non-null     object 
dtypes: bool(2), float64(2), int64(2), object(9)
memory usage: 2.2+ KB


In [11]:
df.describe()

,id,vote_average,vote_count,popularity
count,20.000000,20.000000,20.000000,20.000000
mean,662690.750000,7.065000,1363.450000,2125.279550
std,204158.108956,0.770697,2937.699001,2628.517291
min,335787.000000,5.600000,2.000000,11.469000
25%,508520.750000,6.500000,12.750000,74.893000
50%,657643.000000,7.000000,352.000000,556.723500
75%,821011.000000,7.575000,1168.250000,3010.129750
max,970308.000000,8.700000,12901.000000,8293.346000


In [12]:
genres_url = 'https://api.themoviedb.org/3/genre/movie/list?api_key='+api_key

In [13]:
genres_r = requests.get(genres_url)
genres_r

<Response [200]>

In [14]:
genres_json = genres_r.json()
genres_json

{'genres': [{'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 16, 'name': 'Animation'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 80, 'name': 'Crime'},
  {'id': 99, 'name': 'Documentary'},
  {'id': 18, 'name': 'Drama'},
  {'id': 10751, 'name': 'Family'},
  {'id': 14, 'name': 'Fantasy'},
  {'id': 36, 'name': 'History'},
  {'id': 27, 'name': 'Horror'},
  {'id': 10402, 'name': 'Music'},
  {'id': 9648, 'name': 'Mystery'},
  {'id': 10749, 'name': 'Romance'},
  {'id': 878, 'name': 'Science Fiction'},
  {'id': 10770, 'name': 'TV Movie'},
  {'id': 53, 'name': 'Thriller'},
  {'id': 10752, 'name': 'War'},
  {'id': 37, 'name': 'Western'}]}

In [15]:
genres_json.keys()

dict_keys(['genres'])

In [16]:
genres_df = pd.DataFrame(genres_json['genres'])
genres_df

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


In [17]:
for _, row in genres_df.iterrows():
    df[row["name"]] = df["genre_ids"].apply(lambda x : 1 if row["id"] in x else 0)

In [18]:
df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,poster_path,release_date,title,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,False,/qK7Ssnrfvrt65F66A1thvehfQg2.jpg,"[16, 10751, 35, 12, 14, 9648]",420821,en,Chip 'n Dale: Rescue Rangers,Decades since their successful television seri...,/7UGmn8TyWPPzkjhLUW58cOUHjPS.jpg,2022-05-20,Chip 'n Dale: Rescue Rangers,...,0,0,0,1,0,0,0,0,0,0
1,False,/rl49Qr4KbfTSYjXYAN8fMDIt2xS.jpg,"[28, 878, 14]",526896,en,Morbius,"Dangerously ill with a rare blood disorder, an...",/6JjfSchsU6daXk2AKX8EEBjO3Fm.jpg,2022-03-30,Morbius,...,0,0,0,0,0,1,0,0,0,0
2,False,/kiH3KPWi7BaRMvdAigcwrUFViHl.jpg,"[9648, 53, 28, 80]",818397,en,Memory,"Alex, an assassin-for-hire, finds that he's be...",/tXpLmtnArbGcE75iPJVYibW7BBf.jpg,2022-04-28,Memory,...,0,0,0,1,0,0,0,1,0,0
3,False,/kB1Qrsg189eDz4Ej3ABXtLudClq.jpg,[35],810171,en,The Valet,World famous movie star Olivia faces a PR disa...,/q7FmdJHKMLIC4XgWfcFRIu2iVdL.jpg,2022-05-11,The Valet,...,0,0,0,0,0,0,0,0,0,0
4,False,/wp3vpSWAIjKSEeYb8F5NSZfONqw.jpg,"[28, 12, 878, 35]",545611,en,Everything Everywhere All at Once,An aging Chinese immigrant is swept up in an i...,/w3LxiVYdWWRvEVdn5RYq6jIqkb1.jpg,2022-03-24,Everything Everywhere All at Once,...,0,0,0,0,0,1,0,0,0,0
5,False,/egoyMDLqCxzjnSrWOz50uLlJWmD.jpg,"[28, 878, 35, 10751, 12]",675353,en,Sonic the Hedgehog 2,"After settling in Green Hills, Sonic is eager ...",/6DrHO1jr3qVrViUO6s6kFiAGM7.jpg,2022-03-30,Sonic the Hedgehog 2,...,0,0,0,0,0,1,0,0,0,0
6,False,/xHrp2pq73oi9D64xigPjWW1wcz1.jpg,"[80, 9648, 53]",414906,en,The Batman,"In his second year of fighting crime, Batman u...",/74xTEgt7R36Fpooo50r9T25onhq.jpg,2022-03-01,The Batman,...,0,0,0,1,0,0,0,1,0,0
7,False,/BlC5QrSHnEHFAjAYrLdngUIA8r.jpg,"[35, 99, 28]",828853,en,Jackass 4.5,"Through outrageous, never-before-seen footage,...",/kewHZSdRyOWaLwVwhmoMhjp40Lz.jpg,2022-05-20,Jackass 4.5,...,0,0,0,0,0,0,0,0,0,0
8,False,/6fRVz7ZIZIGFEOq4THY2pdBBhfS.jpg,"[35, 10749]",934761,en,A Perfect Pairing,"To land a major client, an LA wine exec travel...",/h65lLhYEfRGOWVZzwX9n7vYOixf.jpg,2022-05-19,A Perfect Pairing,...,0,0,0,0,1,0,0,0,0,0
9,False,/cqnVuxXe6vA7wfNWubak3x36DKJ.jpg,"[28, 12, 14, 53]",639933,en,The Northman,Prince Amleth is on the verge of becoming a ma...,/zhLKlUaF1SEpO58ppHIAyENkwgw.jpg,2022-04-07,The Northman,...,0,0,0,0,0,0,0,1,0,0


In [19]:
df["Action"]=df["genre_ids"].apply(lambda x : 1 if 28 in x else 0)
df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,poster_path,release_date,title,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,False,/qK7Ssnrfvrt65F66A1thvehfQg2.jpg,"[16, 10751, 35, 12, 14, 9648]",420821,en,Chip 'n Dale: Rescue Rangers,Decades since their successful television seri...,/7UGmn8TyWPPzkjhLUW58cOUHjPS.jpg,2022-05-20,Chip 'n Dale: Rescue Rangers,...,0,0,0,1,0,0,0,0,0,0
1,False,/rl49Qr4KbfTSYjXYAN8fMDIt2xS.jpg,"[28, 878, 14]",526896,en,Morbius,"Dangerously ill with a rare blood disorder, an...",/6JjfSchsU6daXk2AKX8EEBjO3Fm.jpg,2022-03-30,Morbius,...,0,0,0,0,0,1,0,0,0,0
2,False,/kiH3KPWi7BaRMvdAigcwrUFViHl.jpg,"[9648, 53, 28, 80]",818397,en,Memory,"Alex, an assassin-for-hire, finds that he's be...",/tXpLmtnArbGcE75iPJVYibW7BBf.jpg,2022-04-28,Memory,...,0,0,0,1,0,0,0,1,0,0
3,False,/kB1Qrsg189eDz4Ej3ABXtLudClq.jpg,[35],810171,en,The Valet,World famous movie star Olivia faces a PR disa...,/q7FmdJHKMLIC4XgWfcFRIu2iVdL.jpg,2022-05-11,The Valet,...,0,0,0,0,0,0,0,0,0,0
4,False,/wp3vpSWAIjKSEeYb8F5NSZfONqw.jpg,"[28, 12, 878, 35]",545611,en,Everything Everywhere All at Once,An aging Chinese immigrant is swept up in an i...,/w3LxiVYdWWRvEVdn5RYq6jIqkb1.jpg,2022-03-24,Everything Everywhere All at Once,...,0,0,0,0,0,1,0,0,0,0
5,False,/egoyMDLqCxzjnSrWOz50uLlJWmD.jpg,"[28, 878, 35, 10751, 12]",675353,en,Sonic the Hedgehog 2,"After settling in Green Hills, Sonic is eager ...",/6DrHO1jr3qVrViUO6s6kFiAGM7.jpg,2022-03-30,Sonic the Hedgehog 2,...,0,0,0,0,0,1,0,0,0,0
6,False,/xHrp2pq73oi9D64xigPjWW1wcz1.jpg,"[80, 9648, 53]",414906,en,The Batman,"In his second year of fighting crime, Batman u...",/74xTEgt7R36Fpooo50r9T25onhq.jpg,2022-03-01,The Batman,...,0,0,0,1,0,0,0,1,0,0
7,False,/BlC5QrSHnEHFAjAYrLdngUIA8r.jpg,"[35, 99, 28]",828853,en,Jackass 4.5,"Through outrageous, never-before-seen footage,...",/kewHZSdRyOWaLwVwhmoMhjp40Lz.jpg,2022-05-20,Jackass 4.5,...,0,0,0,0,0,0,0,0,0,0
8,False,/6fRVz7ZIZIGFEOq4THY2pdBBhfS.jpg,"[35, 10749]",934761,en,A Perfect Pairing,"To land a major client, an LA wine exec travel...",/h65lLhYEfRGOWVZzwX9n7vYOixf.jpg,2022-05-19,A Perfect Pairing,...,0,0,0,0,1,0,0,0,0,0
9,False,/cqnVuxXe6vA7wfNWubak3x36DKJ.jpg,"[28, 12, 14, 53]",639933,en,The Northman,Prince Amleth is on the verge of becoming a ma...,/zhLKlUaF1SEpO58ppHIAyENkwgw.jpg,2022-04-07,The Northman,...,0,0,0,0,0,0,0,1,0,0


In [20]:
genres_df = pd.DataFrame(genres_json['genres'])
genres_df

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


In [21]:
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns.')

There are 20 rows and 34 columns.


In [22]:
test_url = 'https://api.themoviedb.org/3/trending/movie/week?api_key='+api_key

In [23]:
test_r = requests.get(url)
test_r

<Response [200]>

In [24]:
test_json = test_r.json()
test_json

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/qK7Ssnrfvrt65F66A1thvehfQg2.jpg',
   'genre_ids': [16, 10751, 35, 12, 14, 9648],
   'id': 420821,
   'original_language': 'en',
   'original_title': "Chip 'n Dale: Rescue Rangers",
   'overview': 'Decades since their successful television series was canceled, Chip has succumbed to a life of suburban domesticity as an insurance salesman. Dale, meanwhile, has had CGI surgery and works the nostalgia convention circuit, desperate to relive his glory days. When a former cast mate mysteriously disappears, Chip and Dale must repair their broken friendship and take on their Rescue Rangers detective personas once again to save their friend’s life.',
   'poster_path': '/7UGmn8TyWPPzkjhLUW58cOUHjPS.jpg',
   'release_date': '2022-05-20',
   'title': "Chip 'n Dale: Rescue Rangers",
   'video': False,
   'vote_average': 7.3,
   'vote_count': 36,
   'popularity': 87.881,
   'media_type': 'movie'},
  {'adult': False,
   'backdrop_path': '

In [25]:
test_json.keys()

dict_keys(['page', 'results', 'total_pages', 'total_results'])

In [26]:
test_df = pd.DataFrame(test_json['results'])
test_df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,poster_path,release_date,title,video,vote_average,vote_count,popularity,media_type
0,False,/qK7Ssnrfvrt65F66A1thvehfQg2.jpg,"[16, 10751, 35, 12, 14, 9648]",420821,en,Chip 'n Dale: Rescue Rangers,Decades since their successful television seri...,/7UGmn8TyWPPzkjhLUW58cOUHjPS.jpg,2022-05-20,Chip 'n Dale: Rescue Rangers,False,7.3,36,87.881,movie
1,False,/rl49Qr4KbfTSYjXYAN8fMDIt2xS.jpg,"[28, 878, 14]",526896,en,Morbius,"Dangerously ill with a rare blood disorder, an...",/6JjfSchsU6daXk2AKX8EEBjO3Fm.jpg,2022-03-30,Morbius,False,6.3,938,6323.965,movie
2,False,/kiH3KPWi7BaRMvdAigcwrUFViHl.jpg,"[9648, 53, 28, 80]",818397,en,Memory,"Alex, an assassin-for-hire, finds that he's be...",/tXpLmtnArbGcE75iPJVYibW7BBf.jpg,2022-04-28,Memory,False,7.4,31,118.002,movie
3,False,/kB1Qrsg189eDz4Ej3ABXtLudClq.jpg,[35],810171,en,The Valet,World famous movie star Olivia faces a PR disa...,/q7FmdJHKMLIC4XgWfcFRIu2iVdL.jpg,2022-05-11,The Valet,False,6.3,12,68.677,movie
4,False,/wp3vpSWAIjKSEeYb8F5NSZfONqw.jpg,"[28, 12, 878, 35]",545611,en,Everything Everywhere All at Once,An aging Chinese immigrant is swept up in an i...,/w3LxiVYdWWRvEVdn5RYq6jIqkb1.jpg,2022-03-24,Everything Everywhere All at Once,False,8.7,254,325.871,movie
5,False,/egoyMDLqCxzjnSrWOz50uLlJWmD.jpg,"[28, 878, 35, 10751, 12]",675353,en,Sonic the Hedgehog 2,"After settling in Green Hills, Sonic is eager ...",/6DrHO1jr3qVrViUO6s6kFiAGM7.jpg,2022-03-30,Sonic the Hedgehog 2,False,7.8,1487,6201.709,movie
6,False,/xHrp2pq73oi9D64xigPjWW1wcz1.jpg,"[80, 9648, 53]",414906,en,The Batman,"In his second year of fighting crime, Batman u...",/74xTEgt7R36Fpooo50r9T25onhq.jpg,2022-03-01,The Batman,False,7.8,4672,2950.956,movie
7,False,/BlC5QrSHnEHFAjAYrLdngUIA8r.jpg,"[35, 99, 28]",828853,en,Jackass 4.5,"Through outrageous, never-before-seen footage,...",/kewHZSdRyOWaLwVwhmoMhjp40Lz.jpg,2022-05-20,Jackass 4.5,False,5.6,9,106.149,movie
8,False,/6fRVz7ZIZIGFEOq4THY2pdBBhfS.jpg,"[35, 10749]",934761,en,A Perfect Pairing,"To land a major client, an LA wine exec travel...",/h65lLhYEfRGOWVZzwX9n7vYOixf.jpg,2022-05-19,A Perfect Pairing,False,6.7,25,75.390,movie
9,False,/cqnVuxXe6vA7wfNWubak3x36DKJ.jpg,"[28, 12, 14, 53]",639933,en,The Northman,Prince Amleth is on the verge of becoming a ma...,/zhLKlUaF1SEpO58ppHIAyENkwgw.jpg,2022-04-07,The Northman,False,7.5,974,5772.331,movie


In [27]:
tmdb_5000_credits = pd.read_csv('tmdb_5000_credits.csv', delimiter=',')
nRow, nCol = tmdb_5000_credits.shape
print(f'There are {nRow} rows and {nCol} columns.')

There are 4803 rows and 4 columns.


In [28]:
tmdb_5000_credits

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [29]:
tmdb_5000_credits.describe()

,movie_id
count,4803.000000
mean,57165.484281
std,88694.614033
min,5.000000
25%,9014.500000
50%,14629.000000
75%,58610.500000
max,459488.000000


In [30]:
tmdb_5000_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [31]:
tmdb_5000_movies = pd.read_csv('tmdb_5000_movies.csv', delimiter=',')
nRow, nCol = tmdb_5000_movies.shape
print(f'There are {nRow} rows and {nCol} columns.')

There are 4803 rows and 20 columns.


In [32]:
tmdb_5000_movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [33]:
keywords=display(tmdb_5000_movies.groupby([tmdb_5000_movies.keywords]).size())
pd.set_option('display.max_rows',50)
print(keywords)

keywords
[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                412
[{"id": 100, "name": "slum"}, {"id": 1009, "name": "baby"}, {"id": 1721, "name": "fight"}, {"id": 2483, "name": "nudity"}, {"id": 3077, "name": "leader"}

None


In [34]:
genres=display(tmdb_5000_movies.groupby([tmdb_5000_movies.genres]).size())
pd.set_option('display.max_rows',50)
print(genres)

genres
[]                                                                                                                                                                28
[{"id": 10402, "name": "Music"}, {"id": 10749, "name": "Romance"}]                                                                                                 3
[{"id": 10402, "name": "Music"}, {"id": 12, "name": "Adventure"}, {"id": 16, "name": "Animation"}, {"id": 28, "name": "Action"}, {"id": 35, "name": "Comedy"}]     1
[{"id": 10402, "name": "Music"}, {"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}, {"id": 80, "name": "Crime"}]                                       1
[{"id": 10402, "name": "Music"}, {"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}]                                                                    4
                                                                                                                                                                  ..
[{"

None


temp=display(tmdb_5000_movies.groupby([tmdb_5000_movies.status]).size())
pd.set_option('display.max_rows',50)
print(temp)

In [35]:
status=display(tmdb_5000_movies.groupby([tmdb_5000_movies.status]).size())
pd.set_option('display.max_rows',50)
print(status)

status
Post Production       3
Released           4795
Rumored               5
dtype: int64

None


In [36]:
tmdb_5000_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [37]:
tmdb_5000_movies.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [38]:
print("movies dataset shape: ",tmdb_5000_credits.shape)
print("credits dataset shape: ",tmdb_5000_movies.shape)

movies dataset shape:  (4803, 4)
credits dataset shape:  (4803, 20)


In [39]:
movies=tmdb_5000_movies.merge(tmdb_5000_credits,on="title")
movies.head(10)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,258000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",http://www.sonypictures.com/movies/spider-man3/,559,"[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...",en,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,139.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The battle within.,Spider-Man 3,5.9,3576,559,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
6,260000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://disney.go.com/disneypictures/tangled/,38757,"[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...",en,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The

In [40]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [41]:
movies=movies[["movie_id","genres","title","overview","release_date","keywords","popularity","vote_average","cast","crew","production_companies"]]

In [42]:
movies

,movie_id,genres,title,overview,release_date,keywords,popularity,vote_average,cast,crew,production_companies
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Avatar,"In the 22nd century, a paraplegic Marine is di...",2009-12-10,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",150.437577,7.2,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""name"": ""Ingenious Film Partners"", ""id"": 289..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",2007-05-19,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",139.082615,6.9,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Spectre,A cryptic message from Bond’s past sends him o...,2015-10-26,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",107.376788,6.3,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",The Dark Knight Rises,Following the death of District Attorney Harve...,2012-07-16,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",112.312950,7.6,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""..."
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",John Carter,"John Carter is a war-weary, former military ca...",2012-03-07,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",43.926995,6.1,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]"
...,...,...,...,...,...,...,...,...,...,...,...
4804,9367,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",El Mariachi,El Mariachi just wants to play his guitar and ...,1992-09-04,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",14.269792,6.6,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...","[{""name"": ""Columbia Pictures"", ""id"": 5}]"
4805,72766,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",Newlyweds,A newlywed couple's honeymoon is upended by th...,2011-12-26,[],0.642552,5.9,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",[]
4806,231617,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",2013-10-13,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",1.444476,7.0,"[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","[{""name"": ""Front Street Pictures"", ""id"": 3958}..."
4807,126186,[],Shanghai Calling,When ambitious New York attorney Sam is sent t...,2012-05-03,[],0.857008,5.7,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",[]


In [43]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   movie_id              4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   title                 4809 non-null   object 
 3   overview              4806 non-null   object 
 4   release_date          4808 non-null   object 
 5   keywords              4809 non-null   object 
 6   popularity            4809 non-null   float64
 7   vote_average          4809 non-null   float64
 8   cast                  4809 non-null   object 
 9   crew                  4809 non-null   object 
 10  production_companies  4809 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 450.8+ KB


In [44]:
movies.isnull().sum()

movie_id                0
genres                  0
title                   0
overview                3
release_date            1
keywords                0
popularity              0
vote_average            0
cast                    0
crew                    0
production_companies    0
dtype: int64

In [45]:
movies.dropna(inplace=True)

In [46]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [47]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i["name"])
    return l   

In [48]:
movies["genres"]=movies["genres"].apply(convert)
movies["genres"].head()

0    [Action, Adventure, Fantasy, Science Fiction]
1                     [Adventure, Fantasy, Action]
2                       [Action, Adventure, Crime]
3                 [Action, Crime, Drama, Thriller]
4             [Action, Adventure, Science Fiction]
Name: genres, dtype: object

In [49]:
movies["keywords"]=movies["keywords"].apply(convert)
movies["keywords"].head()

0    [culture clash, future, space war, space colon...
1    [ocean, drug abuse, exotic island, east india ...
2    [spy, based on novel, secret agent, sequel, mi...
3    [dc comics, crime fighter, terrorist, secret i...
4    [based on novel, mars, medallion, space travel...
Name: keywords, dtype: object

In [50]:
movies["production_companies"]=movies["production_companies"].apply(convert)
movies["production_companies"].head()

0    [Ingenious Film Partners, Twentieth Century Fo...
1    [Walt Disney Pictures, Jerry Bruckheimer Films...
2                     [Columbia Pictures, Danjaq, B24]
3    [Legendary Pictures, Warner Bros., DC Entertai...
4                               [Walt Disney Pictures]
Name: production_companies, dtype: object

In [51]:
movies["cast"][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [52]:
def convert3(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            l.append(i["name"])
            counter+=1
        else:
            break
    return l  

In [53]:
movies["cast"]=movies["cast"].apply(convert3)
movies["cast"].head()

0    [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1       [Johnny Depp, Orlando Bloom, Keira Knightley]
2        [Daniel Craig, Christoph Waltz, Léa Seydoux]
3        [Christian Bale, Michael Caine, Gary Oldman]
4      [Taylor Kitsch, Lynn Collins, Samantha Morton]
Name: cast, dtype: object

In [54]:
movies["crew"][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [55]:
def director(obj): 
    l=[]
    for i in ast.literal_eval(obj):
        if i["job"]=="Director":
            l.append(i["name"])
            break
    return l 

In [56]:
movies["crew"]=movies["crew"].apply(director)
movies["crew"].head()

0        [James Cameron]
1       [Gore Verbinski]
2           [Sam Mendes]
3    [Christopher Nolan]
4       [Andrew Stanton]
Name: crew, dtype: object

In [57]:
movies["overview"][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [58]:
movies["overview"]=movies["overview"].apply(lambda x:x.split())
movies["overview"]

0       [In, the, 22nd, century,, a, paraplegic, Marin...
1       [Captain, Barbossa,, long, believed, to, be, d...
2       [A, cryptic, message, from, Bond’s, past, send...
3       [Following, the, death, of, District, Attorney...
4       [John, Carter, is, a, war-weary,, former, mili...
                              ...                        
4804    [El, Mariachi, just, wants, to, play, his, gui...
4805    [A, newlywed, couple's, honeymoon, is, upended...
4806    ["Signed,, Sealed,, Delivered", introduces, a,...
4807    [When, ambitious, New, York, attorney, Sam, is...
4808    [Ever, since, the, second, grade, when, he, fi...
Name: overview, Length: 4805, dtype: object

In [59]:
movies.head()

,movie_id,genres,title,overview,release_date,keywords,popularity,vote_average,cast,crew,production_companies
0,19995,"[Action, Adventure, Fantasy, Science Fiction]",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",2009-12-10,"[culture clash, future, space war, space colon...",150.437577,7.2,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[Ingenious Film Partners, Twentieth Century Fo..."
1,285,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...",2007-05-19,"[ocean, drug abuse, exotic island, east india ...",139.082615,6.9,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[Walt Disney Pictures, Jerry Bruckheimer Films..."
2,206647,"[Action, Adventure, Crime]",Spectre,"[A, cryptic, message, from, Bond’s, past, send...",2015-10-26,"[spy, based on novel, secret agent, sequel, mi...",107.376788,6.3,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[Columbia Pictures, Danjaq, B24]"
3,49026,"[Action, Crime, Drama, Thriller]",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...",2012-07-16,"[dc comics, crime fighter, terrorist, secret i...",112.312950,7.6,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],"[Legendary Pictures, Warner Bros., DC Entertai..."
4,49529,"[Action, Adventure, Science Fiction]",John Carter,"[John, Carter, is, a, war-weary,, former, mili...",2012-03-07,"[based on novel, mars, medallion, space travel...",43.926995,6.1,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],[Walt Disney Pictures]


In [60]:
#removing the space from all the columns values.
movies["genres"]=movies["genres"].apply(lambda x:[i.replace(" ","") for i in x])
movies["keywords"]=movies["keywords"].apply(lambda x:[i.replace(" ","") for i in x])
movies["cast"]=movies["cast"].apply(lambda x:[i.replace(" ","") for i in x])
movies["crew"]=movies["crew"].apply(lambda x:[i.replace(" ","") for i in x])
movies.head()

,movie_id,genres,title,overview,release_date,keywords,popularity,vote_average,cast,crew,production_companies
0,19995,"[Action, Adventure, Fantasy, ScienceFiction]",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",2009-12-10,"[cultureclash, future, spacewar, spacecolony, ...",150.437577,7.2,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Ingenious Film Partners, Twentieth Century Fo..."
1,285,"[Adventure, Fantasy, Action]",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...",2007-05-19,"[ocean, drugabuse, exoticisland, eastindiatrad...",139.082615,6.9,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Walt Disney Pictures, Jerry Bruckheimer Films..."
2,206647,"[Action, Adventure, Crime]",Spectre,"[A, cryptic, message, from, Bond’s, past, send...",2015-10-26,"[spy, basedonnovel, secretagent, sequel, mi6, ...",107.376788,6.3,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Columbia Pictures, Danjaq, B24]"
3,49026,"[Action, Crime, Drama, Thriller]",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...",2012-07-16,"[dccomics, crimefighter, terrorist, secretiden...",112.312950,7.6,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Legendary Pictures, Warner Bros., DC Entertai..."
4,49529,"[Action, Adventure, ScienceFiction]",John Carter,"[John, Carter, is, a, war-weary,, former, mili...",2012-03-07,"[basedonnovel, mars, medallion, spacetravel, p...",43.926995,6.1,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],[Walt Disney Pictures]


In [61]:
movies["tags"]=movies["overview"]+movies["genres"]+movies["keywords"]+movies["cast"]+movies["crew"]
movies["tags"][0]

['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'JamesCameron']

In [62]:
new_df=movies[["movie_id","title","tags"]]
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [63]:
#now we need to again convert tag col values into string 
new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))
new_df.head()

C:\Users\cellville\AppData\Local\Temp\ipykernel_1000\934411879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))


,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [64]:
new_df["tags"]=new_df["tags"].apply(lambda x:x.lower())

C:\Users\cellville\AppData\Local\Temp\ipykernel_1000\4024514323.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"]=new_df["tags"].apply(lambda x:x.lower())


In [65]:
#use stemmer to create the root word
#ps=PorterStemmer()
from nltk.stem import WordNetLemmatizer

wnl = WordNetLemmatizer()

def x(text):
    y=[]
    for i in text.split():
        y.append(wnl.lemmatize(i)) 
    return " ".join(y) 

In [66]:
new_df["tags"]=new_df["tags"].apply(x)
new_df["tags"][0]

LookupError: 
**********************************************************************
  Resource [93momw-1.4[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('omw-1.4')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/omw-1.4[0m

  Searched in:
    - 'C:\\Users\\cellville/nltk_data'
    - 'C:\\Users\\cellville\\anaconda3\\envs\\PythonData\\nltk_data'
    - 'C:\\Users\\cellville\\anaconda3\\envs\\PythonData\\share\\nltk_data'
    - 'C:\\Users\\cellville\\anaconda3\\envs\\PythonData\\lib\\nltk_data'
    - 'C:\\Users\\cellville\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [67]:
cv=CountVectorizer(max_features=5000,stop_words="english")
X_cv=cv.fit_transform(new_df["tags"]).toarray()
X_cv

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [68]:
#tfidf + n gram (also removes stop words)
#1,2 (1 and 2 word combinations)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2),stop_words="english")
X_tfidf = vectorizer.fit_transform(new_df.tags.values)
print(X_tfidf.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [69]:
#pca to vector

In [70]:
# summaalrity score of the ratings (using postlemmetation)
# number of matching words
# number of matching genres/crews/cast
# sentence explaining the movie

# tdif vs count vectorizer: whihc has higher similarity score and use that or average of the both

In [74]:
#calculate the cosine similarity one movie to another movie and get maxium of two arrays + feed into recommendations
# two bc 
#movies name merge and do a comparison
sim_cv=cosine_similarity(X_cv)
sim_tfidf=cosine_similarity(X_tfidf)
print(sim_cv)

[[1.         0.08980265 0.05986843 ... 0.0248452  0.02777778 0.        ]
 [0.08980265 1.         0.06451613 ... 0.02677398 0.         0.        ]
 [0.05986843 0.06451613 1.         ... 0.02677398 0.         0.        ]
 ...
 [0.0248452  0.02677398 0.02677398 ... 1.         0.0745356  0.04774099]
 [0.02777778 0.         0.         ... 0.0745356  1.         0.05337605]
 [0.         0.         0.         ... 0.04774099 0.05337605 1.        ]]


In [75]:
print(sim_tfidf)

[[1.         0.01087611 0.00616873 ... 0.00122235 0.00134975 0.        ]
 [0.01087611 1.         0.00320348 ... 0.00418787 0.         0.        ]
 [0.00616873 0.00320348 1.         ... 0.00379309 0.         0.        ]
 ...
 [0.00122235 0.00418787 0.00379309 ... 1.         0.00627666 0.00776287]
 [0.00134975 0.         0.         ... 0.00627666 1.         0.00397103]
 [0.         0.         0.         ... 0.00776287 0.00397103 1.        ]]


In [76]:
sim_cv.shape

(4805, 4805)

In [77]:
sim_tfidf.shape

(4805, 4805)

In [87]:
# building the movie recommendation function 

def recommend_cv(movie):
    movie_index= new_df[new_df['title']==movie].index[0]
    distance=sim_cv[movie_index]
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [88]:
# lets see the recommendation (seems more accurate)
movies_rec=["Fight Club"]
for i in movies_rec:
    print("If you like:",i)
    print("")
    print("You should watch: \n")
    recommend_cv(i)

If you like: Fight Club

You should watch: 

This Is Martin Bonner
Northfork
Never Back Down
Blue Like Jazz
Fight Valley


In [89]:
# building the movie recommendation function 

def recommend_tfidf(movie):
    movie_index= new_df[new_df['title']==movie].index[0]
    distance=sim_tfidf[movie_index]
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [90]:
# lets see the recommendation 
movies_rec=["Fight Club"]
for i in movies_rec:
    print("If you like:",i)
    print("")
    print("You should watch: \n")
    recommend_tfidf(i)

If you like: Fight Club

You should watch: 

Never Back Down
Raging Bull
Se7en
What's Eating Gilbert Grape
The Machinist


In [ ]:
# two different algorithms
# drop down with option 1 and option 2 examples (cv & tfdif)
# based on genres, casts, keywords and tags
# text processing (nlp)